# Shooting Data Cleaning Notebook

This notebook documents the process I'm using to clean the shooting victimization data Max and Jim obtained.

## Setup

In [1]:
import datetime as dt

import geodatasets
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

from shotspotter import settings

## Source Data

The original source for the shooting victimization data is the Chicago Police Department's "Violence Reduction - Victims of Homicides and Non-Fatal Shootings" dataset on the [Chicago Data Portal](https://data.cityofchicago.org/Public-Safety/Violence-Reduction-Victims-of-Homicides-and-Non-Fa/gumc-mgzr/about_data). Per the description on the portal:

>Each row represents a single victimization, i.e., a unique event when an individual became the victim of a homicide or non-fatal shooting. Each row does not represent a unique victim—if someone is victimized multiple times there will be multiple rows for each of those distinct events.

CPD classified each incident as a fatal or non-fatal shooting using the following methodology:
>A victimization is considered a homicide victimization or non-fatal shooting victimization depending on its presence in CPD's homicide victims data table or its shooting victims data table. A victimization is considered a homicide only if it is present in CPD's homicide data table, while a victimization is considered a non-fatal shooting only if it is present in CPD's shooting data tables and absent from CPD's homicide data table

Importantly, the dataset does not include officer-involved shootings in its definition of homicide, nor does it include cases of involuntary manslaughter or justifiable homicide:
>The definition of “homicide” (shooting or otherwise) does not include justifiable homicide or involuntary manslaughter. This dataset also excludes any cases that CPD considers to be “unfounded” or “noncriminal.” Officer-involved shootings are not included.

In [2]:
shootings = pd.read_csv(
    settings.DATA_DIR_SRC / "Violence_Reduction_-_Victims_of_Homicides_and_Non-Fatal_Shootings_20240905.csv", 
    dtype=str,
)

In [3]:
shootings.head()

,CASE_NUMBER,DATE,BLOCK,VICTIMIZATION_PRIMARY,INCIDENT_PRIMARY,GUNSHOT_INJURY_I,UNIQUE_ID,ZIP_CODE,WARD,COMMUNITY_AREA,...,MONTH,DAY_OF_WEEK,HOUR,LOCATION_DESCRIPTION,STATE_HOUSE_DISTRICT,STATE_SENATE_DISTRICT,UPDATED,LATITUDE,LONGITUDE,LOCATION
0,JF167335,03/08/2022 03:27:00 PM,6000 N KENMORE AVE,HOMICIDE,HOMICIDE,NO,HOM-JF167335-#1,60660,48,EDGEWATER,...,3,3,15,APARTMENT,14,7,02/24/2023 05:55:43 AM,41.9905705,-87.657,POINT (-87.657 41.9905705)
1,A000157,01/01/1996 04:40:00 AM,4300 S STATE ST,HOMICIDE,HOMICIDE,YES,HOM-A000157-#1,60609,3,GRAND BOULEVARD,...,1,2,4,AUTO,6,3,12/21/2010 07:55:15 AM,41.816121654,-87.626845346,POINT (-87.62684534612 41.816121654142)
2,JB409046,08/26/2018 03:35:00 AM,4000 W PALMER ST,BATTERY,BATTERY,YES,SHOOT-JB409046-#1,60639,35,HERMOSA,...,8,1,3,STREET,3,2,03/05/2024 12:10:50 PM,41.920436904,-87.727059059,POINT (-87.727059058699 41.920436904142)
3,JB429086,09/09/2018 10:35:00 PM,3800 W DIVISION ST,BATTERY,BATTERY,YES,SHOOT-JB429086-#1,60651,27,HUMBOLDT PARK,...,9,1,22,STREET,4,2,03/05/2024 12:35:47 PM,41.902326904,-87.721819059,POINT (-87.721819058699 41.902326904142)
4,JD300964,07/17/2020 07:39:00 PM,9800 S AVENUE L,BATTERY,BATTERY,YES,SHOOT-JD300964-#1,60617,10,EAST SIDE,...,7,6,19,SIDEWALK,25,13,03/05/2024 10:13:41 AM,41.717036904,-87.536370941,POINT (-87.536370941301 41.717036904142)


The point of this analysis is to determine whether or not each shooting incident has a corresponding ShotSpotter alert. Per our methodology, the way to do that is to check whether a ShotSpotter alert occurred within 0.5 miles of the incident and within a one-hour window of the incident's location. For further analysis, we also break down shooting incidents into homicides, batteries, and robberies. We also filter out shootings that occurred indoors. For robustness, it would also be nice to have more detailed location info.

That means we need the following information for each shooting:
| Variable | Data Type | Description |
| -------- | --------- | ----------- |
| `id` | `str` | A unique identifier for the shooting |
| `case_number` | `str` | The case number for the shooting |
| `date_time` | `datetime.datetime` | The time and date on which a shooting occurred |
| `latitude` | `np.float64` | The latitude for the shooting |
| `longitude` | `np.float64` | The longitude for the shooting |
| `type` | `pd.Categorical` | Whether the shooting was a homicide, battery, or robbery |
| `indoors` | `bool` | Whether or not the shooting happened indoors — see `shotspotter.settings.INDOOR_LOCATIONS` for list of locations we count as "indoors" |
| `gunshot` | `bool` | Whether or not there was a gunshot injury. |
| `place_description` | `str` | A text description of the place where the crime occurred. |
| `in_coverage_area` | `bool` | Whether or not the incident happened within the ShotSpotter coverage area. This is defined contractually by police district. See [map](https://www.documentcloud.org/documents/24388755-mo-emails_factsheet_guidice?responsive=1&title=1) for details. |
| `police_district` | `pd.Categorical` | The CPD police district in which the incident occurred. |

The relevant columns in the source data are as follows (descriptions and data types from the data dictionary on the Chicago Data Portal):

| Column Name | Data Type | Description |
| ----------- | --------- | ----------- |
| `UNIQUE_ID` | Text | ID unique to each victimization. |
| `CASE_NUMBER` | Text | The Chicago Police Department RD Number (Records Division Number), which is a unique ID assigned to each incident. Due to an incident sometimes involving multiple victimizations, this number is repeated in this dataset for some incidents. |
| `DATE` | Floating timestamp | Date when the victimization occurred. This is sometimes a best estimate. |
| `INCIDENT_PRIMARY` | Text | Text description of the IUCR Code that describes the major crime category it falls into. VICTIMIZATION_PRIMARY represents only the crime that this specific victim within the incident experienced. In rare instances when we were not able to establish whether a non-fatal shooting victimization was an aggravated battery, robbery, or criminal sexual assault, we set this field equal to “NON-FATAL.” |
| `GUNSHOT_INJURY_I` | Text | Indicator field describing whether or not a victim was injured by gunfire. Non-fatal shooting data is not available before 2010 so all non-homicide victimizations will be recorded as “UNKNOWN.” |
| `LATITUDE` | Number | The latitude of the victimization location. In order to preserve anonymity, the given coordinates are not the actual location of the crime. To produce slightly altered coordinates, a circle roughly the size of an average city block was drawn around the original point location, and a new location was picked randomly from a spot around the circumference of that circle. |
| `LONGITUDE` | Number | The longitude of the victimization location. This has been slightly altered to preserve anonymity. (see details under LATITUDE). |
| `LOCATION_DESCRIPTION` | Text | Describes the location where a crime occurred, such as alley, sidewalk, etc. |
| `DISTRICT` | Text | The current CPD district where the victimization occurred, using the boundaries in place as of 11/1/2021. Please refer to the City of Chicago’s Data Portal for a map of districts. |

## Cleaning

The close correspondence between the source columns and the desired output makes cleaning fairly simple. 

### Conversion
First, we convert the raw text data to the appropriate data types. A few explanatory notes:

- Unlike the RFD dataset, the shootings dataset does not use zero-padded numbers for the district labels, so we manually add the zeroes here for consistency.

In [4]:
converted_df = (
    pd.DataFrame(
        {
            "id": shootings["UNIQUE_ID"],
            "case_number": shootings["CASE_NUMBER"],
            "date_time": pd.to_datetime(shootings["DATE"], format="%m/%d/%Y %I:%M:%S %p"),
            "latitude": shootings["LATITUDE"],
            "longitude": shootings["LONGITUDE"],
            "type": pd.Categorical(shootings["INCIDENT_PRIMARY"]),
            "indoors": shootings["LOCATION_DESCRIPTION"].str.strip().isin(settings.INDOOR_LOCATIONS),
            "gunshot": shootings["GUNSHOT_INJURY_I"] == "YES",
            "place_description": shootings["LOCATION_DESCRIPTION"].str.strip(),
            "in_coverage_area": shootings["DISTRICT"].str.pad(width=3, fillchar="0").isin(settings.SHOTSPOTTER_DISTRICTS),
            "police_district": pd.Categorical(shootings["DISTRICT"].str.pad(width=3, fillchar="0")),
        }
    )
    .set_index("id")
)
converted_df.head()

,case_number,date_time,latitude,longitude,type,indoors,gunshot,place_description,in_coverage_area,police_district
id,,,,,,,,,,
HOM-JF167335-#1,JF167335,2022-03-08 15:27:00,41.9905705,-87.657,HOMICIDE,True,False,APARTMENT,False,024
HOM-A000157-#1,A000157,1996-01-01 04:40:00,41.816121654,-87.626845346,HOMICIDE,True,True,AUTO,True,002
SHOOT-JB409046-#1,JB409046,2018-08-26 03:35:00,41.920436904,-87.727059059,BATTERY,False,True,STREET,True,025
SHOOT-JB429086-#1,JB429086,2018-09-09 22:35:00,41.902326904,-87.721819059,BATTERY,False,True,STREET,True,025
SHOOT-JD300964-#1,JD300964,2020-07-17 19:39:00,41.717036904,-87.536370941,BATTERY,False,True,SIDEWALK,True,004


### Filtering

Our analysis only considers shooting incidents in 2023 and 2024, so we need to filter the dataset to only include that time period. We also want to exclude any shootings that happened indoors, as well as homicides that do not involve gunshots and those that fall outside of the ShotSpotter coverage area (defined as the police districts covered by the ShotSpotter contract). Finally, we want to drop any duplicate rows from the dataset (defined as a shooting at the same time and place).

In [5]:
shootings_2023_2024 = (
    converted_df.loc[
        converted_df["date_time"].between(dt.datetime(2023, 1, 1), dt.datetime(2025, 1, 1), inclusive="left")
        & ~converted_df["indoors"]
        & converted_df["gunshot"]
        & converted_df["in_coverage_area"]
    ]
    .drop(columns=["indoors", "gunshot", "in_coverage_area"])
    .drop_duplicates(subset=["date_time", "latitude", "longitude"])
)
shootings_2023_2024.describe()

,date_time
count,2676
mean,2023-11-06 07:38:27.690583040
min,2023-01-01 00:01:00
25%,2023-06-11 21:00:00
50%,2023-10-09 21:46:00
75%,2024-04-28 05:34:30
max,2024-09-01 21:42:00


In [6]:
shootings_2023_2024["type"].value_counts()

type
BATTERY                    1981
HOMICIDE                    598
ROBBERY                      95
NON-FATAL                     2
CRIMINAL SEXUAL ASSAULT       0
Name: count, dtype: int64

### Saving

In [7]:
shootings_2023_2024.to_csv(
    settings.DATA_DIR_PROCESSED / "shooting_victimizations_2023_2024.csv" 
)